# 01 · Preparación de datos (QC + splits + tasks.json)

In [ ]:
# -*- coding: utf-8 -*-
"""
Prepara los splits (train/val/test) de los recorridos del simulador de Udacity
y genera un manifiesto "tasks.json" para el pipeline continual.

Qué hace:
1) Lee el driving_log.csv de cada recorrido (RUNS).
2) Normaliza rutas de imágenes (soporta rutas con barras invertidas y/o comillas).
3) Verifica que existan las imágenes de las tres cámaras (center/left/right).
4) Realiza un split estratificado por rangos de 'steering' para evitar sesgo.
5) Guarda:
   - data/processed/<run>/canonical.csv
   - data/processed/<run>/{train,val,test}.csv
   - data/processed/tasks.json   (orden de tareas y rutas a CSV por split)

Notas:
- La estratificación por bins de 'steering' reduce el sesgo a "recta".
- El seed fija la reproducibilidad del muestreo.
"""

from pathlib import Path
import pandas as pd
import numpy as np
import json

# Raíz del repo (asume que este script vive en notebooks/ o similar)
ROOT = Path.cwd().parent
RAW  = ROOT / "data" / "raw" / "udacity"     # Carpeta con los recorridos originales
PROC = ROOT / "data" / "processed"           # Salidas procesadas

# Lista de recorridos a preparar (debe haber un driving_log.csv en cada uno)
RUNS = ["circuito1", "circuito2"]


def _fix(p: str) -> str:
    """
    Normaliza una ruta de imagen proveniente del CSV.

    - Elimina espacios y comillas al borde.
    - Cambia barras invertidas '\' por barras '/' (compatibilidad Windows→Linux).
    - Si la ruta contiene 'IMG/' (en cualquier mayúscula/minúscula), la recorta
      desde ese punto para dejarla relativa al directorio IMG/.

    Ejemplos:
        r'C:\\data\\udacity\\IMG\\center_1.jpg' -> 'IMG/center_1.jpg'
        'img/LEFT.JPG' -> 'img/LEFT.JPG' (se recorta desde 'img/')
    """
    p = str(p).strip().strip('"').strip("'").replace("\\", "/")
    # Buscamos 'img/' en minúsculas para no depender de mayúsculas/minúsculas
    low = p.lower()
    i = low.rfind("img/")
    if i != -1:
        # Usamos el índice calculado sobre la versión minúscula (misma longitud)
        # para cortar la ruta original preservando su caso original.
        return p[i:]
    return p


def load_csv(csv_path: Path) -> pd.DataFrame:
    """
    Carga un driving_log.csv con los nombres de columnas esperados
    y normaliza las rutas de las tres cámaras.

    driving_log.csv (Udacity) columnas:
      [center, left, right, steering, throttle, brake, speed]
    """
    df = pd.read_csv(
        csv_path,
        header=None,
        names=["center", "left", "right", "steering", "throttle", "brake", "speed"],
    )

    # Normalizamos rutas y las recortamos para que queden relativas a 'IMG/...'
    for cam in ["center", "left", "right"]:
        df[cam] = df[cam].map(_fix)

    return df


def stratified(
    df: pd.DataFrame,
    bins: int = 21,
    train: float = 0.70,
    val: float = 0.15,
    seed: int = 42,
):
    """
    Split estratificado por bins de 'steering' para evitar sesgo.

    Args:
        df: DataFrame con al menos 'steering'.
        bins: número de cortes (p. ej., 21 → 20 rangos). Cuantos más, más fina la estratificación.
        train/val: proporciones de train y val; test se infiere como 1 - train - val.
        seed: fija la reproducibilidad del muestreo.

    Detalles:
        - Calculamos bordes de bining entre min/max de 'steering' (acotando a [-1, 1] por robustez).
        - En cada bin barajamos los índices y cortamos según proporciones.
        - Concatenamos los trozos de cada bin para formar los splits finales.

    Returns:
        (train_df, val_df, test_df)
    """
    # Aseguramos tipo float y acotamos los extremos a [-1, 1] por robustez
    s = df["steering"].astype(float)
    lo = min(float(s.min()), -1.0)
    hi = max(float(s.max()),  1.0)

    # Definimos los bordes de los bins uniformemente en [lo, hi]
    edges = np.linspace(lo, hi, bins)

    # Asignamos a cada fila el índice de bin (0..bins-2); NaN si cae fuera (no debería)
    labels = pd.cut(s, bins=edges, include_lowest=True, labels=False)

    df = df.copy()
    df["_b"] = labels  # columna temporal con el bin asignado

    rng = np.random.default_rng(seed)
    parts = []  # acumulamos (tr, va, te) de cada bin por separado

    # Para cada bin presente en los datos…
    for b in sorted(df["_b"].dropna().unique()):
        g   = df[df["_b"] == b]            # filas del bin b
        idx = np.arange(len(g))            # índices relativos dentro del grupo
        rng.shuffle(idx)                   # barajamos con RNG reproducible

        n   = len(idx)
        ntr = int(round(n * train))
        nva = int(round(n * val))

        # Cortamos (train, val, test) dentro del bin
        parts.append((
            g.iloc[idx[:ntr]],
            g.iloc[idx[ntr : ntr + nva]],
            g.iloc[idx[ntr + nva :]],
        ))

    # Unimos todos los bins para formar los splits finales
    tr = pd.concat([a for a, _, _ in parts], ignore_index=True)
    va = pd.concat([b for _, b, _ in parts], ignore_index=True)
    te = pd.concat([c for _, _, c in parts], ignore_index=True)

    # Limpiamos la columna temporal
    for p in (tr, va, te):
        p.drop(columns=["_b"], inplace=True, errors="ignore")

    return tr, va, te


# --------------------------------------------------------------------------
# Ejecución "script-like": prepara splits por cada run y genera tasks.json
# --------------------------------------------------------------------------
PROC.mkdir(parents=True, exist_ok=True)
manif = []  # recopilación de rutas a splits por cada recorrido

for run in RUNS:
    base = RAW / run
    csv  = base / "driving_log.csv"
    assert csv.exists(), f"No existe el CSV esperado: {csv}"

    # 1) Cargar y normalizar rutas
    df = load_csv(csv)

    # 2) Filtrar filas donde *no* existan imágenes (exigimos las 3 cámaras)
    #    Nota: este filtro es AND: se descarta la fila si falta cualquiera de las tres.
    for cam in ["center", "left", "right"]:
        df = df[(base / df[cam]).apply(lambda p: p.exists())]

    # 3) Guardar una versión canónica del CSV (rutas normalizadas y filtradas)
    out = PROC / run
    out.mkdir(parents=True, exist_ok=True)
    df.to_csv(out / "canonical.csv", index=False)

    # 4) Split estratificado y guardado a disco
    tr, va, te = stratified(df)
    tr.to_csv(out / "train.csv", index=False)
    va.to_csv(out / "val.csv",   index=False)
    te.to_csv(out / "test.csv",  index=False)

    # 5) Añadir al manifiesto para continual (orden y rutas)
    manif.append({
        "run": run,
        "paths": {
            "train": str(out / "train.csv"),
            "val":   str(out / "val.csv"),
            "test":  str(out / "test.csv"),
        },
    })

# 6) tasks.json con el orden y los splits (lo usa 03_TRAIN_EVAL.ipynb)
with open(PROC / "tasks.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            "tasks_order": RUNS,                                   # e.g., ["circuito1", "circuito2"]
            "splits": {m["run"]: m["paths"] for m in manif},       # rutas a CSV por run
        },
        f,
        indent=2,
        ensure_ascii=False,
    )

print("OK:", PROC / "tasks.json")


OK: /home/cesar/proyectos/TFM_SNN/data/processed/tasks.json
